# **💁🏻🗨️💁🏻‍♂️대화 요약 SOLAR API code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 자료에서는 Solar Chat API를 이용하여 대화 요약 대회를 풀어봅니다.     

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

In [ ]:
!pip install openai

In [1]:
import pandas as pd
import os
import time
from tqdm import tqdm
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.
from openai import OpenAI # openai==1.2.0

In [2]:
from dotenv import load_dotenv
from hydra import initialize, compose
import wandb

load_dotenv()
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
# Jupyter 환경에서 실행
initialize(version_base="1.3", config_path="../config", job_name="test_app")  # config_path 지정
cfg = compose(config_name="config")  # config 파일 이름 지정
print(cfg)
os.chdir("../")
runtime_cwd = os.getcwd()  # 현재 작업 디렉터리 가져오기
print(f"runtime_cwd: {runtime_cwd}")
cfg.general.data_path = runtime_cwd + "/data"
cfg.general.model_cache_dir = runtime_cwd + "/models"
cfg.general.output_path = runtime_cwd + "/outputs"

{'prompt': {'mode': 'few-shot', 'version': 'v2', 'use_in_train': False, 'use_in_inference': True, 'n_samples': 2, 'save_predictions': True, 'system': 'You are a expert in the field of dialogue summarization...', 'instruction': 'Following the instructions below...\n'}, 'debug': {'enabled': True, 'train_samples': 20, 'val_samples': 10}, 'prompt_templates': {'1': {'system': 'You are a Korean dialogue summarizer. Follow these rules:\n1. Summarize the dialogue in Korean only\n2. Keep the summary concise and simple \n3. Do not include any English text\n4. Focus on the key points only\n5. Convert colloquial language into formal written language\n', 'instruction': 'Please summarize the following dialogue in Korean.\nBe concise and focus on the key points.\nKeep it under 100 characters.\n', 'few_shot': {'user': '다음 대화를 요약해주세요:\n\nSample Dialogue:\n{sample_dialogue}\n\nSummary:\n', 'assistant': '{sample_summary}\n', 'final_user': 'Dialogue:\n{dialogue}\n\nSummary:\n'}, 'zero_shot': {'user': '다음 

In [3]:

print(f"cfg.keys(): {cfg.keys()}")

# wandb.init(project=cfg.general.wandb.project, entity=cfg.general.wandb.entity, name=cfg.general.timestamp)

# wandb.log({"notebook test": 1})

# wandb.finish()


cfg.keys(): dict_keys(['prompt', 'debug', 'prompt_templates', 'general', 'inference', 'data', 'url', 'metrics', 'huggingface', 'lora', 'common', 'custom_config', 'model', 'train'])


### 2) Solar Chat API Client 생성하기
- 앞으로 Solar Chat API를 사용하기 위해 Client를 생성합니다.

In [4]:
#UPSTAGE_API_KEY = "up_*****************************" # upstage.ai에서 발급받은 API KEY를 입력해주세요.

client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1/solar"
)

### 4) 데이터 불러오기
- 실험에서 쓰일 데이터를 load합니다.

In [59]:
# special token 추가
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gogamza/kobart-base-v2")
special_tokens_dict = {'additional_special_tokens': list(unique_token)}
tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.additional_special_tokens, tokenizer.additional_special_tokens_ids

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


(['#CarNumber#',
  '#PhoneNumber#',
  '#Person4#',
  '#Person6#',
  '#Person7#',
  '#Email#',
  '#Address#',
  '#CardNumber#',
  '#SSN#',
  '#DateOfBirth#',
  '#Person2#',
  '#PassportNumber#',
  '#Person5#',
  '#Person3#',
  '#Person1#',
  '#Person#'],
 [30000,
  30001,
  30002,
  30003,
  30004,
  30005,
  30006,
  30007,
  30008,
  30009,
  30010,
  30011,
  30012,
  30013,
  30014,
  30015])

In [26]:
nouns = get_noun(train_df['dialogue'].iloc[0])


In [ ]:
total_reviews = visualize(nouns,'total') # 워드클라우드 시각화

In [6]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(DATA_PATH,'dev.csv'))
val_df.tail()

NameError: name 'DATA_PATH' is not defined

## 1. Solar Chat API 요약 성능 확인하기
- Solar Chat API을 이용하여 train 및 validation dataset에 포함된 dialogue 샘플을 요약해 봅니다.

In [7]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
rouge = Rouge()
def compute_metrics(pred, gold):
    results = rouge.get_scores(pred, gold, avg=True)
    result = {key: value["f"] for key, value in results.items()}
    return result

In [8]:
# Dialogue를 입력으로 받아, Solar Chat API에 보낼 Prompt를 생성하는 함수를 정의합니다.
def build_prompt(dialogue):
    system_prompt = "You are an expert in the field of dialogue summarization. Please summarize the following dialogue."

    user_prompt = f"Dialogue:\n{dialogue}\n\nSummary:\n"
    
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]

In [9]:
# Solar Chat API를 활용해 Summarization을 수행하는 함수를 정의합니다.
def summarization(dialogue):
    summary = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=build_prompt(dialogue),
    )

    return summary.choices[0].message.content

### (선택) parameter 변경하기
- Solar Chat API를 사용할 때, parameter를 변경하여, 다양한 결과를 얻을 수 있습니다.
- Parameter에 대한 자세한 설명은 [여기](https://developers.upstage.ai/docs/apis/chat#request-body)를 참고해주세요.

In [10]:
def summarization(dialogue):
    summary = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=build_prompt(dialogue),
        temperature=0.2,
        top_p=0.3,
    )

    return summary.choices[0].message.content

Train Dataset을 이용하여 요약이 잘 되는지 확인해 봅니다.

In [11]:
# Train data 중 처음 3개의 대화를 요약합니다.
def test_on_train_data(num_samples=3):
    for idx, row in train_df[:num_samples].iterrows():
        dialogue = row['dialogue']
        summary = summarization(dialogue)
        print(f"Dialogue:\n{dialogue}\n")
        print(f"Pred Summary: {summary}\n")
        print(f"Gold Summary: {row['summary']}\n")
        print("=="*50)

In [ ]:
if __name__ == "__main__":
    test_on_train_data()

Validation Dataset을 이용하여 요약을 진행하고, 성능을 평가해 봅니다.

In [ ]:
# Validation data의 대화를 요약하고, 점수를 측정합니다.
def validate(num_samples=-1):
    val_samples = val_df[:num_samples] if num_samples > 0 else val_df
    
    scores = []
    for idx, row in tqdm(val_samples.iterrows(), total=len(val_samples)):
        dialogue = row['dialogue']
        summary = summarization(dialogue)
        results = compute_metrics(summary, row['summary'])
        avg_score = sum(results.values()) / len(results)
        
        scores.append(avg_score)
        
    val_avg_score = sum(scores) / len(scores)

    print(f"Validation Average Score: {val_avg_score}")

In [ ]:
if __name__ == "__main__":
    validate(100) # 100개의 validation sample에 대한 요약을 수행합니다.
    
    # 전체 validation data에 대한 요약을 수행하고 싶은 경우 아래와 같이 실행합니다.
    # validate() 

## 2. Solar Chat API로 요약하기
- Solar Chat API을 이용하여 test dataset에 포함된 dialogue를 요약하고 제출용 파일을 생성합니다.

In [ ]:
def inference():
    test_df = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

    summary = []
    start_time = time.time()
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
        dialogue = row['dialogue']
        summary.append(summarization(dialogue))
        
        # Rate limit 방지를 위해 1분 동안 최대 100개의 요청을 보내도록 합니다.
        if (idx + 1) % 100 == 0:
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            if elapsed_time < 60:
                wait_time = 60 - elapsed_time + 5
                print(f"Elapsed time: {elapsed_time:.2f} sec")
                print(f"Waiting for {wait_time} sec")
                time.sleep(wait_time)
            
            start_time = time.time()
    
    output = pd.DataFrame(
        {
            "fname": test_df['fname'],
            "summary" : summary,
        }
    )
    
    if not os.path.exists(RESULT_PATH):
        os.makedirs(RESULT_PATH)
    output.to_csv(os.path.join(RESULT_PATH, "output_solar.csv"), index=False)

    return output

In [ ]:
if __name__ == "__main__":
    output = inference()

In [ ]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

## 3. Prompt Engineering
- Prompt engineering을 통해 요약 성능 향상을 시도합니다.

In [ ]:
# Few-shot prompt를 생성하기 위해, train data의 일부를 사용합니다.
few_shot_samples = train_df.sample(1)

sample_dialogue1 = few_shot_samples.iloc[0]['dialogue']
sample_summary1 = few_shot_samples.iloc[0]['summary']

print(f"Sample Dialogue1:\n{sample_dialogue1}\n")
print(f"Sample Summary1: {sample_summary1}\n")

In [ ]:
# Prompt를 생성하는 함수를 수정합니다.
def build_prompt(dialogue):
    system_prompt = "You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue."

    user_prompt = (
        "Following the instructions below, summarize the given document.\n"
        "Instructions:\n"
        "1. Read the provided sample dialogue and corresponding summary.\n"
        "2. Read the dialogue carefully.\n"
        "3. Following the sample's style of summary, provide a concise summary of the given dialogue.\n\n"
        "Sample Dialogue:\n"
        f"{sample_dialogue1}\n\n"
        "Sample Summary:\n"
        f"{sample_summary1}\n\n"
        "Dialogue:\n"
        f"{dialogue}\n\n"
        "Summary:\n"
    )
    
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]

In [ ]:
# 변경된 prompt를 사용하여, train data 중 처음 3개의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    test_on_train_data()

In [ ]:
# 변경된 prompt를 사용하여, validation data의 대화를 요약하고, 점수를 측정합니다.
if __name__ == "__main__":
    validate(100)

다른 방식으로 Few-shot sample을 제공하여 Prompt를 구성해 봅니다.

In [ ]:
# Few-shot sample을 다른 방식으로 사용하여 prompt를 생성합니다.
def build_prompt(dialogue):
    system_prompt = "You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue."

    few_shot_user_prompt_1 = (
        "Following the instructions below, summarize the given document.\n"
        "Instructions:\n"
        "1. Read the provided sample dialogue and corresponding summary.\n"
        "2. Read the dialogue carefully.\n"
        "3. Following the sample's style of summary, provide a concise summary of the given dialogue. Be sure that the summary is simple but captures the essence of the dialogue.\n\n"
        "Dialogue:\n"
        f"{sample_dialogue1}\n\n"
        "Summary:\n"
    )
    few_shot_assistant_prompt_1 = sample_summary1
    
    user_prompt = (
        "Dialogue:\n"
        f"{dialogue}\n\n"
        "Summary:\n"
    )
    
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": few_shot_user_prompt_1},
        {"role": "assistant", "content": few_shot_assistant_prompt_1},
        {"role": "user", "content": user_prompt},
    ]

In [ ]:
# 변경된 prompt를 사용하여, train data 중 처음 3개의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    test_on_train_data()

In [ ]:
# 변경된 prompt를 사용하여, validation data의 대화를 요약하고, 점수를 측정합니다.
if __name__ == "__main__":
    validate(100)

### (선택) 변경된 Prompt로 test dataset에 대한 요약을 진행합니다.
- 변경된 prompt를 통해 점수가 개선되었다면, test dataset에 대한 요약을 진행하고 제출합니다.

In [ ]:
# 변경된 prompt를 사용하여, test data의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    output = inference()

In [ ]:
output